In [ ]:
import os

# Change directory to cadmesh repo until conda/pip deployment is ready
os.chdir("/home/sebastian/Projects/ABC/cadmesh/")

from cadmesh import process_step_folder

results = process_step_folder(data_path="data/abc_0000", output_path="data/results/abc_0000", log_path="data/log_abc_0000", file_pattern="*/*.step", file_range=[800, -1])

In [2]:
print(sum(results))

import glob
from tqdm.auto import tqdm
import datetime
logs = sorted(glob.glob("data/log_conv/*.log"))

times = []
parts = []
errors = []
elogs = []
n_errors = []
n_elogs = []
success = []

pbar = tqdm(range(len(logs)))
for i in pbar:
    #pbar.set_description("Processing %s"%logs[i])
    log = logs[i]
    with open(log, "r") as fi:
        lines = fi.readlines()
        
    if len(lines) < 2:
        errors.append(i)
        elogs.append("%i: %s"%(i, "Lines missing"))
        continue
    time_start = " ".join(lines[0].split(" ")[:2])[:-4]
    time_end = " ".join(lines[-1].split(" ")[:2])[:-4]
    #for line in lines:
    t_s = datetime.datetime.strptime(time_start, '%Y-%m-%d %H:%M:%S')
    t_e = datetime.datetime.strptime(time_end, '%Y-%m-%d %H:%M:%S')
    times.append((t_e - t_s).total_seconds())
    
    try:
        parts.append(int(lines[1].split(" ")[5]))
    except:
        parts.append(0)
        errors.append(i)
        elogs.append("%i: %s"%(i, "Translation problem"))
    
    for line in lines:
        if "ERROR" in line and "Nurbs conversion error" in line:
            n_errors.append(i)
            n_elogs.append("%i: %s"%(i, line))
        elif "ERROR" in line and not "Nurbs conversion error" in line:
            errors.append(i)
            elogs.append("%i: %s"%(i, line))
            
        if "Stat dict: Done" in line:
            success.append(i)


956


In [19]:
from yaml import CLoader
import yaml

def parse_geometry_files(path):
    files = glob.glob(path)
    print(len(files))
    curve_types = []
    surf_types = []
    for file in tqdm(files):
        try:
            with open(file, "r") as fi:
                parts = yaml.load(fi, Loader=CLoader)

            if type(parts) == dict:
                for p in parts["parts"]:
                    for c3d in p["3dcurves"]:
                        curve_types.append(c3d["type"])
                    for s3d in p["surfaces"]:
                        surf_types.append(s3d["type"])
        except:
            continue
    
    return curve_types, surf_types
                
ct, st = parse_geometry_files("data/results_conv/*_geo.yaml")

992


In [32]:
counts_c = {"Line": 0, "BSpline": 0, "Circle": 0, "Ellipse": 0, "Other": 0}
for c in ct:
    counts_c[c] += 1
    
counts_s = {"Plane": 0, "Cylinder": 0, "Cone": 0, "Torus": 0, "Other": 0, "Revolution": 0, "BSpline": 0, "Sphere": 0, "Extrusion": 0, "Offset": 0}
for c in st:
    counts_s[c] += 1
print(counts_c, counts_s)

{'Line': 291341, 'BSpline': 188704, 'Circle': 189288, 'Ellipse': 10865, 'Other': 3711} {'Plane': 113428, 'Cylinder': 75372, 'Cone': 17282, 'Torus': 11986, 'Other': 0, 'Revolution': 3671, 'BSpline': 30706, 'Sphere': 3491, 'Extrusion': 2605, 'Offset': 136}
